<a href="https://colab.research.google.com/github/22ananya/MUSI6201/blob/main/Final%20Project%20-%20Stereo%20Demixing/Run_SI_SDR_Eval_on_Saved_Mixes_OU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import generally required packages - update as needed

In [1]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as sp
import scipy.io.wavfile as wav

Import/Install the prerequisite code for implementing the Cadenza challenge - includes baselines, other important files

In [2]:
!pip install pyclarity==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 17.5 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=

Import the dataset for the Cadenza challenge directly through the Google Drive link - Only needs to be done once! So now commented out


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check current path

In [4]:
import os
print(os.getcwd())

/content


Change current path to Cadenza folder

In [5]:
os.chdir('/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation')
print(os.getcwd())

/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation


#Process single audio file through the entire model step by step

Import all dependencies - same as enhance.py file

In [6]:
from __future__ import annotations

import json
import logging
from pathlib import Path

# pylint: disable=import-error
import hydra
import numpy as np
import torch
from numpy import ndarray
from omegaconf import DictConfig
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB

from clarity.enhancer.compressor import Compressor
from clarity.enhancer.nalr import NALR
from clarity.evaluator.haaqi import compute_haaqi
from clarity.utils.audiogram import Audiogram, Listener
from clarity.utils.file_io import read_signal
from clarity.utils.flac_encoder import FlacEncoder
from clarity.utils.results_support import ResultsFile
from clarity.utils.signal_processing import (
    clip_signal,
    denormalize_signals,
    normalize_signal,
    resample,
    to_16bit,
    compute_rms,
    resample,
)
from clarity.utils.source_separation_support import get_device, separate_sources
from recipes.cad_icassp_2024.baseline.evaluate import (
    apply_gains,
    apply_ha,
    make_scene_listener_list,
    remix_stems,
    load_reference_stems,
)

logger = logging.getLogger(__name__)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/evaluate.py:190: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import all the required functions defined in Enhance.py that do not need to be changed

In [7]:
from recipes.cad_icassp_2024.baseline.enhance import (
    save_flac_signal,
    decompose_signal,
    process_remix_for_listener
)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/enhance.py:182: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import the correct config file (hardcoded location)

In [8]:
from omegaconf import OmegaConf
config = OmegaConf.load('config.yaml')
#print("default source sep model: ", config.separator.model)

Set input directory

In [9]:
reference_folder = Path("Val_Ref_Tracks")
enhanced_mix_folder = Path("Val_enhanced_signals_openunmix_100") # select openunmix or demucs as needed

# load the audio files in provided path

In [10]:
enhanced_files = [] # initialize list of enhanced audio files
reference_files = [] # initialize list of reference files
for file in os.listdir(enhanced_mix_folder): # iterate over all files in the directory
    if file.endswith('.flac'): # if the file is an audio file
        enhanced_files.append(os.path.join(enhanced_mix_folder, file)) # add the file to the list of audio files
        reference_files.append(os.path.join(reference_folder, file.replace('.flac','_ref.flac'))) # add the corresponding ref file to the list of ref files

In [11]:
print(enhanced_files[0])
print(reference_files[0])
print(len(enhanced_files))

Val_enhanced_signals_openunmix_100/scene_50001_L5048_remix.flac
Val_Ref_Tracks/scene_50001_L5048_remix_ref.flac
100


Load listener data

In [12]:
config.path.root = '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/Validation'


In [13]:
# Load listener audiograms and songs
listener_dict = Listener.load_listener_dict(config.path.listeners_file)

with Path(config.path.gains_file).open("r", encoding="utf-8") as file:
    gains = json.load(file)

with Path(config.path.scenes_file).open("r", encoding="utf-8") as file:
    scenes = json.load(file)

with Path(config.path.scene_listeners_file).open("r", encoding="utf-8") as file:
    scenes_listeners = json.load(file)

with Path(config.path.music_file).open("r", encoding="utf-8") as file:
    songs = json.load(file)

In [ ]:
#print(songs)

Create a list of songs, listeners (audiogram) and head position (hrtf) to generate, or evaluate the mix - based on provided data

In [14]:
# Select a batch to process
scene_listener_pairs = make_scene_listener_list(
    scenes_listeners, config.evaluate.small_test
)

scene_listener_pairs = scene_listener_pairs[
    config.evaluate.batch :: config.evaluate.batch_size
]

In [15]:
print(scene_listener_pairs[0])

('scene_50001', 'L5020')


# Load functions required for audio evaluation

Add functions and variables needed for evaluation and scoring

In [25]:
scores_headers = [
    "scene",
    "song",
    "listener",
    "left_score",
    "right_score",
    "score",
]


results_file = ResultsFile(
            "Val_scores_openunmix_SI_SDR.csv",
            header_columns=scores_headers,
)

# Process Audio - Currently set to process a fixed number of runs (listener - scene pairings, can be changed to run entire dataset)

# Implementing SI - SDR eval

In [26]:
from torchmetrics.audio import ScaleInvariantSignalDistortionRatio
si_sdr = ScaleInvariantSignalDistortionRatio()
previous_song = ""
num_tracks = len(enhanced_files)

for idx in range(100):
    # Extract track, listener, scene info from file name
    file_name = enhanced_files[idx]
    sname = file_name.split("/", 1)[1].split("_L",1)[0]
    lname = "L" +  file_name.split("/", 1)[1].split("_L",1)[1].split("_")[0]

    # Get the listener's audiogram
    listener = listener_dict[lname]
    scene = scenes[sname]
    song_name = f"{scene['music']}-{scene['head_loudspeaker_positions']}"

    print(sname)
    print(lname)

    # load audio files for HAAQI
    reference_mixture = read_signal(
            filename=reference_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    enhanced_mixture = read_signal(
            filename=enhanced_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    # Evaluate - compare with the generated mixes
    # Compute the scores
    if np.argmax([max(np.shape(reference_mixture)), max(np.shape(enhanced_mixture))]):
        enhanced_mixture = enhanced_mixture[:max(np.shape(reference_mixture)), :]
    else:
        reference_mixture = reference_mixture[:max(np.shape(enhanced_mixture)), :]

    target = torch.from_numpy(reference_mixture)
    preds = torch.from_numpy(enhanced_mixture)


    left_score = si_sdr(preds[:,0], target[:,0]) # SI SDR
    right_score = si_sdr(preds[:,1], target[:,1]) # SDR
    #score = np.mean([left_score, right_score])
    print(np.mean([left_score, right_score]))

        # Save scores
    results_file.add_result(
        {
            "scene": sname,
            "song": song_name,
            "listener": listener.id,
            "left_score": left_score,
            "right_score": right_score,
            "score": float(np.mean([left_score, right_score])),
        }
    )


scene_50001
L5048
17.618298
scene_50002
L5011
13.322449
scene_50002
L5024
12.78653
scene_50003
L5027
10.816305
scene_50003
L5053
11.538066
scene_50004
L5009
8.080107
scene_50004
L5013
6.4432335
scene_50005
L5091
14.274176
scene_50005
L5005
13.471987
scene_50006
L5047
11.665165
scene_50006
L5031
7.922909
scene_50007
L5024
8.650057
scene_50007
L5018
8.25575
scene_50008
L5007
11.615641
scene_50008
L5086
14.928133
scene_50009
L5066
13.239706
scene_50009
L5018
11.754055
scene_50010
L5064
7.206939
scene_50010
L5068
7.0713587
scene_50011
L5072
11.085505
scene_50011
L5086
10.460188
scene_50012
L5053
13.327613
scene_50012
L5045
12.523432
scene_50013
L5071
16.355839
scene_50013
L5037
13.032964
scene_50014
L5066
9.09388
scene_50014
L5057
7.7448177
scene_50015
L5062
9.440943
scene_50015
L5092
8.516012
scene_50016
L5092
7.5312243
scene_50016
L5051
8.228328
scene_50017
L5053
8.0024185
scene_50017
L5093
6.3608904
scene_50018
L5091
19.676397
scene_50018
L5027
19.42412
scene_50019
L5027
17.930805
scene

LibsndfileError: ignored